## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
data = results["data"]["stations"]

In [4]:
len(data)

666

In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head(5)

,capacity,eightd_has_key_dispenser,lat,lon,name,region_id,rental_methods,short_name,station_id
0,39,False,40.767272,-73.993929,W 52 St & 11 Ave,71.0,"[KEY, CREDITCARD]",6926.01,72
1,33,False,40.719116,-74.006667,Franklin St & W Broadway,71.0,"[KEY, CREDITCARD]",5430.08,79
2,27,False,40.711174,-74.000165,St James Pl & Pearl St,71.0,"[KEY, CREDITCARD]",5167.06,82
3,62,False,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,71.0,"[KEY, CREDITCARD]",4354.07,83
4,39,False,40.741776,-74.001497,W 17 St & 8 Ave,71.0,"[KEY, CREDITCARD]",6148.02,116


In [6]:
# We drop the 'rental methods' column, as it contains multiple values and 
#we cannot insert lists in a dataase cell.
df.drop('rental_methods', axis=1, inplace = True)
df.head(5)

,capacity,eightd_has_key_dispenser,lat,lon,name,region_id,short_name,station_id
0,39,False,40.767272,-73.993929,W 52 St & 11 Ave,71.0,6926.01,72
1,33,False,40.719116,-74.006667,Franklin St & W Broadway,71.0,5430.08,79
2,27,False,40.711174,-74.000165,St James Pl & Pearl St,71.0,5167.06,82
3,62,False,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,71.0,4354.07,83
4,39,False,40.741776,-74.001497,W 17 St & 8 Ave,71.0,6148.02,116


### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [7]:
!sudo -H pip3 install -U sqlalchemy

Requirement already up-to-date: sqlalchemy in /usr/local/lib/python3.5/dist-packages


In [8]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'localhost', 
    user = 'root',
    password = 'dwdstudent2015')

engine = create_engine(conn_string)
con = engine.connect()

Once we have connected successfully, we need to create our database:

In [9]:
# Query to create a database
db_name = 'citibike_stations'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
engine.execute(create_db_query)
# And lets switch to the database
engine.execute("USE {db}".format(db=db_name))

/usr/local/lib/python3.5/dist-packages/sqlalchemy/engine/default.py:470: Warning: (1007, "Can't create database 'citibike_stations'; database exists")
  cursor.execute(statement, parameters)


Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [10]:
# This step is typically optional, but it is good practice to define explicitly the 
# data types before storing things in a database. In many cases, this can be ommitted, though.
import sqlalchemy
dtype = {
    'capacity': sqlalchemy.types.SMALLINT(),
    'eightd_has_key_dispenser':  sqlalchemy.types.BOOLEAN,
    'lat': sqlalchemy.types.Float, 
    'lon': sqlalchemy.types.Float,
    'name': sqlalchemy.types.VARCHAR(50),
    'region_id': sqlalchemy.types.VARCHAR(5),
    'short_name': sqlalchemy.types.VARCHAR(10),
    'station_id': sqlalchemy.types.SMALLINT()
}

In [11]:
table_name = 'Stations'
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
df.to_sql(name = table_name, schema = db_name, con = engine, if_exists = 'replace', index = False, dtype = dtype)

In [12]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = 'ALTER TABLE Stations ADD PRIMARY KEY(station_id)'
engine.execute(add_key_query)

### Reading from a SQL Database in Python using Pandas

We can similarly read from the database using Pandas

In [13]:
query =  "SELECT * FROM Stations"
df2 = pd.read_sql(query, con=engine)
df2.head(5)

,capacity,eightd_has_key_dispenser,lat,lon,name,region_id,short_name,station_id
0,39,0,40.7673,-73.9939,W 52 St & 11 Ave,71,6926.01,72
1,33,0,40.7191,-74.0067,Franklin St & W Broadway,71,5430.08,79
2,27,0,40.7112,-74.0002,St James Pl & Pearl St,71,5167.06,82
3,62,0,40.6838,-73.9763,Atlantic Ave & Fort Greene Pl,71,4354.07,83
4,39,0,40.7418,-74.0015,W 17 St & 8 Ave,71,6148.02,116


And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [14]:
# The necessary library to write in Excel
!sudo pip3 install -U xlwt

The directory '/home/ubuntu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/ubuntu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
Requirement already up-to-date: xlwt in /usr/local/lib/python3.5/dist-packages


In [15]:
df2.to_excel("citibike.xls")
df2.to_csv("citibike.csv")

### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.